In [ ]:
#MNIST prediction with FC layers, and IMAGE Data generator 

In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import os
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
# set image dimension for Conv layer etc based on tensor flow or theano
K.set_image_dim_ordering('tf')

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# load (downloaded if needed to : C:\Users\sidha\.keras\datasets\mnist.npz) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [ ]:
print(X_train.shape) # X_train.shape result is a tuple
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# number of training samples
N1 = X_train.shape[0]  # same as  N1= X_train.shape and then N1 = N1[0]
N2 = X_test.shape[0]  
h = X_train.shape[1]
w = X_train.shape[2]



In [ ]:
#Baseline Model with Multi-Layer Perceptrons

#For a multi-layer perceptron model we must reduce the images down into a vector of pixels. 
#In this case the 28×28 sized images will be 784 pixel input values.
num_pixels = h*w
# reshape N1 samples to num_pixels
x_train = X_train.reshape(N1, num_pixels).astype('float32') # shape is now (60000,784)
x_test = X_test.reshape(N2, num_pixels).astype('float32') # shape is now (10000,784)

In [ ]:
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
#Finally, the output variable is an integer from 0 to 9. This is a multi-class classification problem. 10 digits 
# classified to 10 classes
#As such, it is good practice to use a one hot encoding of the class values,
#transforming the vector of class integers into a binary matrix.

#We can easily do this using the built-in np_utils.to_categorical() helper function in Keras.
y_train = np_utils.to_categorical(y_train) #(60000,10): 10000 lables for 10 classes
y_test = np_utils.to_categorical(y_test) # (10000,10): 10000 lables for 10 classes
num_classes = y_test.shape[1]


In [ ]:
def baseline_model():
# create model
    model = Sequential()
    # Define input layerwhich is same as hidden layer with the same number of neurons as there are inputs (784). 
    # use RELU for this hidden layer
    #model.add(Dense(784,input_shape=(N1,w,h,1), kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    # Define output layer with softmax function
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    #  use ADAm optimizer and  Logarithmic loss or  categorical_crossentropy Loss
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = baseline_model()
model.summary()

In [ ]:
batch_size = 200
epochs = 20
max_batches = 2 * len(x_train) / batch_size # 2*60000/200

In [ ]:
# reshape to be [samples][width][height][ channel] for ImageDataGenerator
x_t = X_train.reshape(N1, w, h, 1).astype('float32')
datagen = ImageDataGenerator(rescale= 1./255)
train_gen = datagen.flow(x_t, y_train, batch_size=batch_size)
for e in range(epochs):
    batches = 0
    for x_batch, y_batch in train_gen:
    # x_batch is of size [batch_sz,w,h,ch]: resize to [bth_sz,pixel_sz]: (200,28,28,1)-> (200,784)
    # for model.fit
        x_batch = np.reshape(x_batch, [-1, num_pixels])            
        model.fit(x_batch, y_batch,validation_split=0.15,verbose=0)
        batches += 1
        print("Epoch %d/%d, Batch %d/%d" % (e+1, epochs, batches, max_batches))
        if batches >= max_batches:
        # we need to break the loop by hand because
        # the generator loops indefinitely
            break
    


In [ ]:
# Final evaluation of the model on test data( will predict to classes and also give error)
scores = model.evaluate(x_test, y_test, verbose=0)
print("Baseline Accuracy: %.2f%%" % (scores[1]*100))
print("Baseline Error: %.2f%%" % (100-scores[1]*100))